# Maha Ashraf Alkhairy 
## Data Mining HW 2: 

## Written Part: 
### Question 1.a: 

Items: a, b, c, d, e, f

pass 1: set size 1: <br/>
     C<sub>k</sub> = {a}:5, {b}:3, {c}:6, {d}:5, {e}:4, {f}:4 = L<sub>k</sub> (all are frequent) 

pass 2: set size 2:   
     C<sub>k</sub> = {a, b}: 1 , {a, c}: 2 , {a, d}: 2, {a, e}: 2 , {a, f}: 1,
                    {b, c}: 3, {b, d}: 1, {b, e}: 0, {b, f}: 2,
                    {c, d}: 2, {c, e}: 3, {c, f}: 3, 
                    {d, e}: 1, {d, f}: 3, 
                    {e, f}: 1 <br/> L<sub>k</sub> = {b, c}: 3, {c, e}: 3, {c, f}: 3, {d, f}: 3
     
pass 3: set size 3: <br/>
    C<sub>k</sub> = {{b, c, e}: 0, {c, e, f}: 1, {c, d, f}: 2, {b, c, f}: 2 <br/> L<sub>k</sub> = [] (empty no more frequent item sets)

maximal frequent item sets: {a}, {b,c}, {c, e}, {c, f}, {d, f}
   







        
   
        

 1.b) 
     chose {c, f} from the maximal sets  and the association rule that satisfies the constraints (support/frequency at least 0.3 and confidence at least 0.6) is f -> c as shown in the table below: 
        
| association rule       | support                      | confidence                         |
|:----------------------:|:----------------------------:|:----------------------------------:|
| c -> f                 | freq(c, f)/N = 3 / 10 = 0.3  | freq(c, f)/freq(c) = 3 / 6 = 0.5   |
| f -> c                 | freq(c, f)/N = 3 / 10 = 0.3  | freq(c, f)/freq(f) = 3 / 4 = 0.75 |

### Question 2.a: 
--------------------------
calculate frequency of items of size 1: 
    {a}:8 , {b}:6 , {c}:6 , {d}:4, {e}:4, {f}:1 --> remove f from the transactions since not frequent then draw the full FP-tree

##### Full FP-tree: 


<img src="FP_full.jpg" style="width: 500px;"/> 




---------

### Question 2.b: 

<img src="FP_d_subtree.jpg" style="width: 300px;"/> 

----------------------------------------------------------

<img src="FP_tree_conditional.jpg" style="width: 200px;"/> 

----------------------------------------------------------

<img src="FP_xd.jpg" style="width: 300px;"/> 

----------------------------------------------------------
  

##### Based on the d's conditional FP-tree the frequent patterns are: 
  {d}, {b, d}, {c, d}, {a, b, d}, {a, c, d}, {a, b, c, d}
  

# PROGRAMMING PART

In [1]:
## imports for FP_Growth algorithm
import pyspark 
from pyspark import SparkContext
from pyspark.mllib.fpm import FPGrowth

# to parse and order outputs
from operator import itemgetter
import re

In [2]:
SparkContext.setSystemProperty('spark.executor.memory', '2g')
sc = pyspark.SparkContext('local[*]')

In [3]:
rdd = sc.textFile('publications.txt')

In [4]:
rdd_prev = sc.textFile('AP_train.txt')

## Question 1:

The differences between the AP_train file and the Publications file syntactically is as follows: <br/>
  1) The ordering of the lines are different: the publication index is placed between the venue and references. <br/>
  2) There are empty references in publications.txt and not in AP_train.txt <br/>
  3) There are no spaces after the special characters (ones that indecate whether it is a publication index, ....) in publication.txt whereas AP_train.txt has spaces <br/>
  4) authors are , seperated in publications.txt and ; seperated in AP_train.txt

-------------------------------------------------------------------------

The differences in terms of statistics:  (publications.txt, AP_train.txt)

| statistic              | publications.txt             | AP_train.txt                       |
|:----------------------:|:----------------------------:|:----------------------------------:|
| publications           |      2146341                 |    1976815                         |
| authors                | 1232494                      | 1478733                            |
| venues                 | 8707                         | 255685                             |
| references             |   528263                     |     871089                         | 
| years                  |       80                     |             69                     |
  

In [5]:
## get the rdd for the specific items in AP_train.txt
index_rdd_prev = rdd_prev.filter(
                lambda l: re.match('^#index(.*)', l)).map(
                lambda l: re.match('^#index(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [6]:
venue_rdd_prev = rdd_prev.filter(
                lambda l: re.match('^#c(.*)', l)).map(
                lambda l: re.match('^#c(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [7]:
ref_rdd_prev = rdd_prev.filter(
                lambda l: re.match('^#%(.*)', l)).map(
                lambda l: re.match('^#%(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [8]:
yop_rdd_prev = rdd_prev.filter(
                lambda l: re.match('^#t(.*)', l)).map(
                lambda l: re.match('^#t(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [9]:
## get the rdd for the specific things in publications.txt
index_rdd = rdd.filter(
                lambda l: re.match('^#index(.*)', l)).map(
                lambda l: re.match('^#index(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [10]:
author_rdd = rdd.filter(
                lambda l: re.match('^#@(.*)', l)).map(
                lambda l: re.match('^#@(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [11]:
venue_rdd = rdd.filter(
                lambda l: re.match('^#c(.*)', l)).map(
                lambda l: re.match('^#c(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [12]:
ref_rdd = rdd.filter(
                lambda l: re.match('^#%(.*)', l)).map(
                lambda l: re.match('^#%(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [13]:
yop_rdd = rdd.filter(
                lambda l: re.match('^#t(.*)', l)).map(
                lambda l: re.match('^#t(.*)', l).group(1).strip()).filter(
                lambda l: l != "")

In [14]:
## counting for publications.txt
unique_index_pub_count = index_rdd.distinct().count()

In [16]:
unique_venue_count = venue_rdd.distinct().count()

In [17]:
unique_yop_count = yop_rdd.distinct().count()

In [18]:
unique_ref_count = ref_rdd.distinct().count()

In [19]:
print("number of unique publications in publications.txt: ", 
      unique_index_pub_count)
## parsed later in the file when getting venues associated with authors
# print("number of unique authors in publications.txt: ", unique_author_count)
print("number of unique venues in publications.txt: ", 
      unique_venue_count)
print("number of unique years in publications.txt: ", 
      unique_yop_count)
print("number of unique references in publications.txt: ",
      unique_ref_count)

number of unique publications in publications.txt:  2146341
number of unique venues in publications.txt:  8707
number of unique years in publications.txt:  80
number of unique references in publications.txt:  528263


In [20]:
## counting for AP_train.txt
unique_index_pub_count_prev = index_rdd_prev.distinct().count()


In [21]:
unique_venue_count_prev = venue_rdd_prev.distinct().count()

In [22]:
unique_yop_count_prev = yop_rdd_prev.distinct().count()

In [23]:
unique_ref_count_prev = ref_rdd_prev.distinct().count()

In [24]:
print("number of unique publications in AP_train.txt: ", unique_index_pub_count_prev)
# print("number of unique authors in AP_train.txt: ", unique_author_count_prev)
print("number of unique venues in AP_train.txt: ", unique_venue_count_prev)
print("number of unique years in AP_train.txt: ", unique_yop_count_prev)
print("number of unique references in AP_train.txt: ", unique_ref_count_prev)

number of unique publications in AP_train.txt:  1976815
number of unique venues in AP_train.txt:  255685
number of unique years in AP_train.txt:  69
number of unique references in AP_train.txt:  871089


## Question 2: 

In [25]:
transactions = author_rdd.map(lambda l: list(set(l.strip().split(','))))

In [26]:
## support threshold 1e-4
model_1 = FPGrowth.train(transactions, minSupport=1e-4,  numPartitions=1)
result_1 = model_1.freqItemsets().collect()


In [27]:
for i in result_1[:10]:
    print(i)

FreqItemset(items=['Christos H. Papadimitriou'], freq=355)
FreqItemset(items=['Fumiyuki Adachi'], freq=227)
FreqItemset(items=['Hao Wang'], freq=348)
FreqItemset(items=['Christian S. Jensen'], freq=302)
FreqItemset(items=['Bernd Freisleben'], freq=226)
FreqItemset(items=['Kishor S. Trivedi'], freq=298)
FreqItemset(items=['Mubarak Shah'], freq=225)
FreqItemset(items=['David Zhang'], freq=390)
FreqItemset(items=['Hui Liu'], freq=248)
FreqItemset(items=['Lei Chen'], freq=380)


In [29]:
## support threshold 1e-5
model_2 = FPGrowth.train(transactions, minSupport=1e-5,  numPartitions=1)
result_2 = model_2.freqItemsets().collect()

In [30]:
for i in result_2[:10]: 
    print(i)

FreqItemset(items=['Philip Levis'], freq=58)
FreqItemset(items=['Myung-Sun Baek'], freq=24)
FreqItemset(items=['Mark J. Karol'], freq=26)
FreqItemset(items=['Xiang Ji'], freq=23)
FreqItemset(items=['Ork de Rooij'], freq=22)
FreqItemset(items=['Gaoxi Xiao'], freq=32)
FreqItemset(items=['Leslie G. Valiant'], freq=109)
FreqItemset(items=['Martin Margala'], freq=81)
FreqItemset(items=['Jason Dykes'], freq=30)
FreqItemset(items=['Satya Dharanipragada'], freq=23)


In [32]:
## support threshold 0.5e-5
model_3 = FPGrowth.train(transactions, minSupport=0.5e-5,  numPartitions=1)
result_3 = model_3.freqItemsets().collect()

In [39]:
for i in result_3[:10]: 
    print(i)

FreqItemset(items=['Wei Wang'], freq=1293)
FreqItemset(items=['Wei Zhang'], freq=856)
FreqItemset(items=['Lei Zhang'], freq=841)
FreqItemset(items=['Wei Li'], freq=805)
FreqItemset(items=['H. Vincent Poor'], freq=735)
FreqItemset(items=['Jun Wang'], freq=717)
FreqItemset(items=['Philip S. Yu'], freq=711)
FreqItemset(items=['Wen Gao'], freq=707)
FreqItemset(items=['Thomas S. Huang'], freq=691)
FreqItemset(items=['Lei Wang'], freq=690)


In [34]:
## support threshold 1e-6 , does not work with large data (runs out of memory)
# model_4 = FPGrowth.train(transactions, minSupport=1e-6,  numPartitions=1)
# result_4 = model_5.freqItemsets().collect()

### Question 2a: 
When we succesively decrease the support threshold, the number of frequent items and the co-occurence of the authors increases to the point of not being able to handle the calculation with the heap memory. 

In [35]:
## for the authors
print("number of frequent items when the support threshold is 1e-4: ", len(result_1))
print("number of frequent items when the support threshold is 1e-5: ", len(result_2))
print("number of frequent items when the support threshold is 0.5e-5: ", len(result_3))
# print("number of frequent items when the support threshold is 1e-6: ", len(result_4))

number of frequent items when the support threshold is 1e-4:  665
number of frequent items when the support threshold is 1e-5:  58955
number of frequent items when the support threshold is 0.5e-5:  168269


In [14]:
def get_coocurrences(frequent_items, item, num_top): 
    """
    gets the items that frequently coocur with the given item
    :frequent_items: [[string, ...], ...]
    :item: string
    :num_top: int
    """
    lst = filter(lambda r: item in r[0], frequent_items)
    top_x = []
    for l in lst:
        for i in l[0]: 
            if len(top_x) == num_top: 
                break
            if i == item: 
                continue 
            if i in top_x: 
                continue
            else: 
                top_x.append(i)
    return top_x
    
def print_cooccurences(colist): 
    """
    print the items in the list seperated by a new line 
    """
    for i in colist: 
        print("\t", i)
    

### Question 2b: 

In [49]:
# when the support threshold is 0.5e-5
author = ['Rakesh Agrawal', 'Jiawei Han', 'Zoubin Ghahramani', 'Christos Faloutsos']
result_3.sort(key=itemgetter(1), reverse=True)

coauthors_1 = get_coocurrences(result_3, author[0], 5)
coauthors_2 = get_coocurrences(result_3, author[1], 5)
coauthors_3 = get_coocurrences(result_3, author[2], 5)
coauthors_4 = get_coocurrences(result_3, author[3], 5)


print("The top 5 coauthors of Rakesh Agrawal ordered by frequency: ") 
print_cooccurences(coauthors_1)
print("\n -----------") 
print("The top 5 coauthors of Jiawei Han ordered by frequency: ") 
print_cooccurences(coauthors_2)
print("\n -----------")
print("The top 5 coauthors of Zoubin Ghahramani ordered by frequency: ")
print_cooccurences(coauthors_3)
print("\n -----------") 
print("The top 5 coauthors of  Christos Faloutsos ordered by frequency: ")
print_cooccurences(coauthors_4)
print("\n -----------")
        


The top 5 coauthors of Rakesh Agrawal ordered by frequency: 
	 Ramakrishnan Srikant
	 Jerry Kiernan
	 H. V. Jagadish
	 Yirong Xu
	 Michael J. Carey

 -----------
The top 5 coauthors of Jiawei Han ordered by frequency: 
	 Xifeng Yan
	 Philip S. Yu
	 Jian Pei
	 Yizhou Sun
	 Xin Jin

 -----------
The top 5 coauthors of Zoubin Ghahramani ordered by frequency: 
	 David L. Wild
	 Katherine A. Heller
	 Michael I. Jordan

 -----------
The top 5 coauthors of  Christos Faloutsos ordered by frequency: 
	 Hanghang Tong
	 Spiros Papadimitriou
	 Jimeng Sun
	 Caetano Traina Jr.
	 Agma J. M. Traina

 -----------


## Question 3

In [4]:
transactions_by_author = sc.textFile("venue_baskets.txt").map(lambda l: list(set(l.strip().split(' ; ')))).filter(lambda s: s != "")

In [5]:
model_ven_1 = FPGrowth.train(transactions_by_author, minSupport=1e-3,  numPartitions=1)
result_ven_1 = model_ven_1.freqItemsets().collect()

In [6]:
model_ven_2 = FPGrowth.train(transactions_by_author, minSupport=0.4e-3,  numPartitions=1)

In [7]:
result_ven_2 = model_ven_2.freqItemsets().collect()

In [ ]:
# model_ven_3 = FPGrowth.train(transactions_by_author, minSupport=1e-4,  numPartitions=1)

In [ ]:
# does not work on large data sets (memory error)
# result_ven_3 = model_ven_3.freqItemsets().collect()

In [57]:
for i in result_ven_1[:10]: 
    print(i)

FreqItemset(items=['ICSLP'], freq=4022)
FreqItemset(items=['ICSLP', 'EUROSPEECH'], freq=1660)
FreqItemset(items=['ICSLP', 'INTERSPEECH'], freq=1366)
FreqItemset(items=['Ad Hoc Networks'], freq=1632)
FreqItemset(items=['Journal of Chemical Information and Computer Sciences'], freq=3871)
FreqItemset(items=['Ann. Math. Artif. Intell.'], freq=1588)
FreqItemset(items=['SIAM J. Comput.'], freq=3000)
FreqItemset(items=['SIAM J. Comput.', 'Inf. Process. Lett.'], freq=1249)
FreqItemset(items=['SIAM J. Comput.', 'Theor. Comput. Sci.'], freq=1349)
FreqItemset(items=['SIAM J. Comput.', 'CoRR'], freq=1455)


In [8]:
for i in result_ven_2[:10]: 
    print(i)

FreqItemset(items=['ICSLP'], freq=4022)
FreqItemset(items=['ICSLP', 'EUROSPEECH'], freq=1660)
FreqItemset(items=['ICSLP', 'EUROSPEECH', 'INTERSPEECH'], freq=929)
FreqItemset(items=['ICSLP', 'INTERSPEECH'], freq=1366)
FreqItemset(items=['ICSLP', 'ICASSP'], freq=504)
FreqItemset(items=['J. Computer-Mediated Communication'], freq=1070)
FreqItemset(items=['IEEE Communications Surveys and Tutorials'], freq=797)
FreqItemset(items=['Ad Hoc Networks'], freq=1632)
FreqItemset(items=['Ad Hoc Networks', 'GLOBECOM'], freq=585)
FreqItemset(items=['Ad Hoc Networks', 'ICC'], freq=539)


In [ ]:
# for i in result_ven_3[:10]: 
#     print(i)

### Question 3a
When we decreas the support threshold the number of frequent items inceases and that co-occurence of venues increases sometime to the point where the heap memory cannot handle it. 

In [9]:
print("number of frequent item sets with 1e-3: ", len(result_ven_1))
print("number of frequent item sets with 0.4e-3: ", len(result_ven_2))
# print("number of frequent item sets with 1e-4: ", len(result_ven_3))

number of frequent item sets with 1e-3:  981
number of frequent item sets with 0.4e-3:  4359


### Question 3b


In [15]:
ml = "NIPS"
dm = "KDD"
db = "VLDB"
cn = "INFOCOM"
nlp = "ACL"

result_ven_2.sort(key=itemgetter(1), reverse=True)

top_venues_ml = get_coocurrences(result_ven_2, ml, 10)
top_venues_dm = get_coocurrences(result_ven_2, dm, 10)
top_venues_db = get_coocurrences(result_ven_2, db, 10)
top_venues_cn = get_coocurrences(result_ven_2, cn, 10)
top_venues_nlp = get_coocurrences(result_ven_2, nlp, 10)


# venue_groups_ml = filter(lambda r: ml in r[0], result_ven_2) 
# venue_groups_dm = filter(lambda r: dm in r[0], result_ven_2) 
# venue_groups_db = filter(lambda r: db in r[0], result_ven_2) 
# venue_groups_cn = filter(lambda r: cn in r[0], result_ven_2) 
# venue_groups_nlp = filter(lambda r: nlp in r[0], result_ven_2) 


# def get_the_top_10_associated_ven(lst, ven): 
#     """
#     :lst: [['string', ...], ...] 
#     """
#     top = []
#     for l in lst:
#         for v in l[0]: 
#             if len(top) == 10: 
#                 break
#             if v == ven: 
#                 continue 
#             if v in top: 
#                 continue
#             else: 
#                 top.append(v)
#     return top


print("for Machine learning: the top ten publications that authors also publish in are: ") 
print_cooccurences(top_venues_ml)
print("\n--------------")

print("for data mining: the top ten publications that authors also publish in are: ") 
print_cooccurences(top_venues_dm)
print("\n--------------")

print("for data bases: the top ten publications that authors also publish in are: ")
print_cooccurences(top_venues_db)
print("\n--------------")
      
print("for computer networks: the top ten publications that authors also publish in are: ")
print_cooccurences(top_venues_cn)
print("\n--------------")
      
print("for natural language processing: the top ten publications that authors also publish in are: ") 
print_cooccurences(top_venues_nlp)
print("\n--------------")

for Machine learning: the top ten publications that authors also publish in are: 
	 CoRR
	 ICML
	 Neural Computation
	 Journal of Machine Learning Research - Proceedings Track
	 Journal of Machine Learning Research
	 IEEE Trans. Pattern Anal. Mach. Intell.
	 CVPR
	 Neurocomputing
	 Neural Networks
	 IJCAI

--------------
for data mining: the top ten publications that authors also publish in are: 
	 CoRR
	 ICDM
	 CIKM
	 IEEE Trans. Knowl. Data Eng.
	 SDM
	 ICML
	 WWW

--------------
for data bases: the top ten publications that authors also publish in are: 
	 ICDE
	 SIGMOD Conference
	 CoRR
	 IEEE Trans. Knowl. Data Eng.
	 SIGMOD Record
	 EDBT
	 CIKM
	 IEEE Data Eng. Bull.
	 VLDB J.
	 ACM Trans. Database Syst.

--------------
for computer networks: the top ten publications that authors also publish in are: 
	 GLOBECOM
	 ICC
	 CoRR
	 IEEE/ACM Trans. Netw.
	 IEEE Journal on Selected Areas in Communications
	 Computer Networks
	 Computer Communications
	 ICDCS
	 IEEE Trans. Parallel Distri